In [2]:
import os
import numpy as np
import dask.array as da
import metpy.constants as mpconsts
import metpy.units as units
from func import cal_buoyancy  # import the function from your script
from pathlib import Path
import pandas as pd
from func import cal_buoyancy
from sklearn.metrics import mean_squared_error
import glob
import os
import re
import traceback
from scipy.interpolate import interp1d
import requests
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
from pydoc import help
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk

In [4]:
backup_dir = Path.home() / "backup_data"

backup_dir.mkdir(parents=True, exist_ok=True)

# path to interpolated humidity, pressure, and temperature 

file_path_main_arrays = backup_dir / "backup_arrays_for_buoyancy.npz"

# path to times, latitudes and longitudes

file_path_metadata = backup_dir / "backup_metadata_for_buoyancy.npz"

data = np.load(file_path_main_arrays, mmap_mode='r')

print("file containing arrays loaded")

interpolated_pressure = data["array1"]

print("interpolated_pressure loaded") 

interpolated_q = data["array2"]

print("interpolated_q loaded") 

interpolated_t = data["array3"]

print("interpolated_t loaded")

interpolated_qfit = data["array_4"]

print("interpolated_qfit loaded") 

metadata = np.load(file_path_metadata)

times = metadata['times']

lats = metadata['lats']

lons = metadata['lons']

interpolated_altitudes = np.arange(0, 4000 + 20, 20)  

print("All arrays successfully loaded!")

file containing arrays loaded
interpolated_pressure loaded
interpolated_q loaded
interpolated_t loaded
interpolated_qfit loaded
All arrays successfully loaded!


In [11]:
split_size = 306 # 2 years for each split

def split_array(array, split_size):
    
    time_splits = array.shape[0] // split_size
    
    return np.array_split(array, time_splits, axis=0)

split_interpolated_pressure = split_array(interpolated_pressure, split_size)

split_interpolated_q = split_array(interpolated_q, split_size)

split_interpolated_t = split_array(interpolated_t, split_size)

split_interpolated_qfit = split_array(interpolated_qfit, split_size)

split_times = split_array(times, split_size)

print(f"Number of splits for interpolated_pressure: {len(split_interpolated_pressure)}")

print(f"Number of splits for interpolated_q: {len(split_interpolated_q)}")

print(f"Number of splits for interpolated_t: {len(split_interpolated_t)}")

print(f"Number of splits for interpolated_qfit: {len(split_interpolated_qfit)}")


Number of splits for interpolated_pressure: 9
Number of splits for interpolated_q: 9
Number of splits for interpolated_t: 9
Number of splits for interpolated_qfit: 9


In [17]:
backup_dir = os.path.expanduser("~/split_SGP_domain_arrays") 

os.makedirs(backup_dir, exist_ok=True)  

year_folders = [
    
    "2001-2002", "2003-2004", "2005-2006", "2007-2008", "2009-2010",
    
    "2011-2012", "2013-2014", "2015-2016", "2017-2018"
]

arrays_to_save = {
    
    'interp_pres_small': split_interpolated_pressure,
    
    'interp_temp_small': split_interpolated_t,
    
    'interp_q_small': split_interpolated_q,
    
    'interp_qfit_small': split_interpolated_qfit,
    
    'times': split_times,
    
    'interpolated_altitudes': interpolated_altitudes, 
    
    'lats': lats, 
    
    'lons': lons
}

for folder_name, pres_split in zip(year_folders, split_interpolated_pressure):
    
    folder_path = os.path.join(backup_dir, folder_name)
    
    os.makedirs(folder_path, exist_ok=True)  

    file_path = os.path.join(folder_path, f"arrays_for_buoyancy_{folder_name}.npz")

    np.savez_compressed(file_path, 
                        
                        interp_pres_small=split_interpolated_pressure[year_folders.index(folder_name)],
                        
                        interp_temp_small=split_interpolated_t[year_folders.index(folder_name)],
                        
                        interp_q_small=split_interpolated_q[year_folders.index(folder_name)],
                        
                        interp_qfit_small=split_interpolated_qfit[year_folders.index(folder_name)],
                        
                        times=split_times[year_folders.index(folder_name)],  

                        interpolated_altitudes=interpolated_altitudes,  
                        
                        lats=lats, 
                        
                        lons=lons
                       )

    print(f"Arrays saved to {file_path}")

print("Saved files:", os.listdir(backup_dir))


Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2001-2002/arrays_for_buoyancy_2001-2002.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2003-2004/arrays_for_buoyancy_2003-2004.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2005-2006/arrays_for_buoyancy_2005-2006.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2007-2008/arrays_for_buoyancy_2007-2008.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2009-2010/arrays_for_buoyancy_2009-2010.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2011-2012/arrays_for_buoyancy_2011-2012.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2013-2014/arrays_for_buoyancy_2013-2014.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2015-2016/arrays_for_buoyancy_2015-2016.npz
Arrays saved to /home/annierosen16/split_SGP_domain_arrays/2017-2018/arrays_for_buoyancy_2017-2018.npz
Saved files: ['2001-2002', '2003-2004', '2005-2006', '2007-2008', '2009-2